# 📂 Dataset Loading & Preprocessing

In this section, we load all station-level datasets, merge them into a unified structure, clean missing values, convert timestamps, and prepare them for feature engineering.

## Topics Covered
- Loading CSV files for each station & Assembling a dictionary of datasets (`get_datasets`)
- Cleaning and normalizing feature columns (`clean_datasets`)
- Feature engineering (`make_features`)
- Adding frost/temperature targets (`add_targets`)
- Building inputs and outputs for models to fit on (`build_xy`)

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd 
import lightgbm as lgb
import matplotlib.pyplot as plt

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    brier_score_loss,
    mean_absolute_error,
    confusion_matrix,
    roc_curve,
    precision_recall_curve,
    auc,
)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

# Root directory for plots
PLOTS_ROOT = Path("plots")
PLOTS_ROOT.mkdir(exist_ok=True)

# Root directory for models
MODELS_ROOT = Path("models")
MODELS_ROOT.mkdir(exist_ok=True)

In [2]:
# Path on CIMIS Datasets [Custom]
DATA_DIR = "../cimis-hourly-data-multiple-stations/"

def get_datasets() -> dict: 
    !touch buffer.tmp
    !(ls $DATA_DIR | grep "csv") > buffer.tmp
    name_to_data = dict() # Dictionary for Name to Data
    # Read the buffer.tmp and create Name to Data pairs
    with open("buffer.tmp", "r") as file:
        contents = [f for f in file.read().split("\n") if f]
        for content in contents:
            file_name = content.split(".")[0]
            if "all" not in file_name:
                name_to_data[file_name] = pd.read_csv(DATA_DIR + content)
            
        !rm -f buffer.tmp
        return name_to_data

def clean_datasets(datasets: dict) -> dict:
    for _, df in datasets.items():
        df.drop(columns=[col for col in df.columns if "qc" in col], inplace=True)
        df.rename(columns=lambda c: c.replace(" ", "_"), inplace=True)
    return datasets

In [3]:
def make_features(df, H):
    df = df.copy()

    # Detect single-station vs multi-station mode
    has_stn = "Stn_Id" in df.columns
    n_stations = df["Stn_Id"].nunique() if has_stn else 1
    multi_station = has_stn and n_stations > 1

    if multi_station:
        # Multi-station: groupby
        g = df.groupby("Stn_Id", group_keys=False)

        def roll(col, w):
            return (
                g[col]
                .rolling(w)
                .min()
                .reset_index(level=0, drop=True)
            )

        def diff(col, w):
            return g[col].diff(w)

    else:
        # Single-station: no groupby needed
        
        def roll(col, w):
            return df[col].rolling(w).min()

        def diff(col, w):
            return df[col].diff(w)

    # ---------- Simple per-row features (no groupby needed) ----------
    # Dewpoint depression: closer = more humid
    df["dewpoint_dep"] = df["Air_Temp_(C)"] - df["Dew_Point_(C)"]
    # Is dim sunlight
    df["is_dim"] = (df["Sol_Rad_(W/sq.m)"] < 50).astype(int)
    # Is very calm winds
    df["is_very_calm"] = (df["Wind_Speed_(m/s)"] < 1.0).astype(int)

    # Is winter based on Julian day
    jd = pd.to_numeric(df["Jul"], errors="coerce").fillna(0)  # Julian Day
    WINTER_START = 305   # Nov 1
    WINTER_END   =  74   # Mar 15
    df["is_winter"] = ((jd >= WINTER_START) | (jd <= WINTER_END)).astype(int)

    # Hour/day cyclical encodings
    hour = pd.to_numeric(df["Hour_(PST)"], errors="coerce").fillna(0) % 24
    hour_angle = 2 * np.pi * hour / 24.0

    day = pd.to_numeric(df["Jul"], errors="coerce").fillna(0) % 365
    day_angle = 2 * np.pi * day / 365.0

    df["hour_sin"] = np.sin(hour_angle)
    df["hour_cos"] = np.cos(hour_angle)
    df["day_sin"] = np.sin(day_angle)
    df["day_cos"] = np.cos(day_angle)

    # ---------- Time-series features (roll/diff via helpers) ----------
    tfs = [3, 6, 12, 24]
    for time in tfs:
        # Air Temperature
        df[f"air_temp_roll_min_{time}h"] = roll("Air_Temp_(C)", time)
        df[f"air_temp_change_{time}h"]   = diff("Air_Temp_(C)", time)

        # Dew Point
        df[f"dew_point_roll_min_{time}h"] = roll("Dew_Point_(C)", time)
        df[f"dew_point_change_{time}h"]   = diff("Dew_Point_(C)", time)

        # Wind Speed
        df[f"wind_spd_roll_min_{time}h"] = roll("Wind_Speed_(m/s)", time)
        df[f"wind_spd_change_{time}h"]   = diff("Wind_Speed_(m/s)", time)

        # Solar Radiation
        df[f"sol_rad_roll_min_{time}h"] = roll("Sol_Rad_(W/sq.m)", time)
        df[f"sol_rad_change_{time}h"]   = diff("Sol_Rad_(W/sq.m)", time)

    return df

def add_targets(df, H):
    df[f"frost_{H}h"] = (df["Air_Temp_(C)"].shift(-H) < 0).astype(int)
    df[f"temp_{H}h"]  =  df["Air_Temp_(C)"].shift(-H)
    return df

def build_xy(df_raw, H, feature_cols=None):
    # Build rebuild df
    df = df_raw.copy()
    df = make_features(df, H)
    df = add_targets(df, H)
    
    # Targets
    frost_col = f"frost_{H}h"
    temp_col  = f"temp_{H}h"
    # Columns to Ignore
    ignore_cols = ['Stn_Id','Stn_Name','CIMIS_Region', 'Date', 'Hour_(PST)', 'Jul']

    # Filter for Feature Cols
    if feature_cols is None:
        is_valid_feature_col = lambda c: c in df.columns and c not in [frost_col, temp_col, *ignore_cols]
        feature_cols = [c for c in df.columns if is_valid_feature_col(c)]

    # X (Features) and Y (Targets)
    needed = feature_cols + [frost_col, temp_col]
    df = df.dropna(subset=[c for c in needed if c in df.columns])

    X = df[feature_cols]
    y_frost = df[frost_col].astype(int)
    y_temp  = df[temp_col].astype(float)
    return X, y_frost, y_temp, feature_cols

# 📊 Metrics & Visualization Tools

This section introduces the tools used to evaluate calibration, discrimination, and regression performance across both teacher and student models. It includes scalar metrics, curve-based evaluation tools, horizon summaries, and LOSO spatial generalization plots.

## Metrics Included
- ROC–AUC-CURVE – discrimination score for ranking  
- PR–AUC_CUR – ranking quality for rare frost events  
- Brier Score – probability calibration and sharpness  
- ECE (Expected Calibration Error) – bucketed calibration measure  
- MAE (Temperature) – regression accuracy for predicted temperature  
- Confusion Matrix (TP, FP, TN, FN)

---

## Visualization & Metric Helper Functions

- **plot_roc_curves**  
  Plots ROC curves for multiple models using a dictionary of probability outputs and reports AUC for each model.

- **plot_pr_curves**  
  Plots Precision–Recall curves for multiple models and computes Average Precision (AP / PR–AUC), useful for rare-event evaluation.

- **plot_reliability_curves**  
  Generates calibration diagrams by binning predicted probabilities and comparing predicted vs. observed outcome frequencies.

- **compute_ece**  
  Computes Expected Calibration Error (ECE) by aggregating the weighted accuracy–confidence gap over probability bins.

- **plot_metric_bar**  
  Produces bar charts for any scalar metric across models (e.g., Brier, MAE, ECE) using a `{model_name: value}` dictionary.

- **plot_horizon_summaries**  
  Plots metric performance across forecasting horizons (e.g., 3h, 6h, 12h, 24h), generating one summary chart per model × metric.

- **plot_loso_summary**  
  Visualizes LOSO cross-station generalization using mean ± standard deviation bar charts across metrics (Brier, ECE, ROC–AUC, PR–AUC, MAE).

In [4]:
def plot_roc_curves(y_true, prob_dict, title=None, save_path=None):
    fig, ax = plt.subplots(figsize=(5, 5))
    for label, probs in prob_dict.items():
        fpr, tpr, _ = roc_curve(y_true, probs)
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr, label=f"{label} (AUC={roc_auc:.3f})")
    ax.plot([0, 1], [0, 1], "k--", linewidth=1)
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title(title or "ROC curves")
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.tight_layout()
    if save_path is not None:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_path, dpi=150, bbox_inches="tight")
    else:
        plt.show()
    plt.close(fig)


def plot_pr_curves(y_true, prob_dict, title=None, save_path=None):
    fig, ax = plt.subplots(figsize=(5, 5))
    for label, probs in prob_dict.items():
        precision, recall, _ = precision_recall_curve(y_true, probs)
        ap = average_precision_score(y_true, probs)
        ax.plot(recall, precision, label=f"{label} (AP={ap:.3f})")
    ax.set_xlabel("Recall")
    ax.set_ylabel("Precision")
    ax.set_title(title or "Precision–Recall curves")
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.tight_layout()
    if save_path is not None:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_path, dpi=150, bbox_inches="tight")
    else:
        plt.show()
    plt.close(fig)


def plot_reliability_curves(y_true, prob_dict, n_bins=10, title=None, save_path=None):
    fig, ax = plt.subplots(figsize=(5, 5))
    for label, probs in prob_dict.items():
        prob_true, prob_pred = calibration_curve(
            y_true, probs, n_bins=n_bins, strategy="uniform"
        )
        ax.plot(prob_pred, prob_true, marker="o", linewidth=1, label=label)
    ax.plot([0, 1], [0, 1], "k--", linewidth=1)
    ax.set_xlabel("Predicted probability")
    ax.set_ylabel("Observed frequency")
    ax.set_title(title or "Reliability diagram")
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.tight_layout()
    if save_path is not None:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_path, dpi=150, bbox_inches="tight")
    else:
        plt.show()
    plt.close(fig)

def plot_metric_bar(metric_name, metric_dict, title=None, save_path=None):
    labels = list(metric_dict.keys())
    values = [metric_dict[k] for k in labels]
    fig, ax = plt.subplots(figsize=(5, 4))
    x = np.arange(len(labels))
    ax.bar(x, values)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=15)
    ax.set_ylabel(metric_name)
    ax.set_title(title or metric_name)
    ax.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    if save_path is not None:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_path, dpi=150, bbox_inches="tight")
    else:
        plt.show()
    plt.close(fig)
    
def plot_horizon_summaries(metrics_over_H, metrics_to_plot):
    for model_name, by_H in metrics_over_H.items():

        model_dir = PLOTS_ROOT / str(model_name)
        model_dir.mkdir(parents=True, exist_ok=True)

        for metric_name in metrics_to_plot:

            # Extract {H: metric_value}
            metric_series = {
                H: m[metric_name]
                for H, m in by_H.items()
                if metric_name in m
            }

            if not metric_series:
                continue

            # Sort horizons (so bars go in correct order)
            horizons = sorted(metric_series.keys())   # e.g. [3, 6, 12, 24]
            values   = [metric_series[H] for H in horizons]

            # Bar plot
            fig, ax = plt.subplots(figsize=(6, 4))
            x = np.arange(len(horizons))
            ax.bar(x, values)

            # Label each bar with its corresponding H
            ax.set_xticks(x)
            ax.set_xticklabels([f"{H}h" for H in horizons])

            ax.set_ylabel(metric_name)
            ax.set_title(f"{model_name} {metric_name} vs Horizon")
            ax.grid(axis="y", alpha=0.3)

            # Add value text on each bar
            for i, v in enumerate(values):
                ax.text(
                    i,
                    v,
                    f"{v:.3f}",
                    ha="center",
                    va="bottom",
                    fontsize=9
                )

            plt.tight_layout()

            # Save file
            save_path = model_dir / f"{metric_name}_vs_H.png"
            save_path.parent.mkdir(parents=True, exist_ok=True)
            fig.savefig(save_path, dpi=150, bbox_inches="tight")
            plt.close(fig)



def plot_loso_summary(summary, title=None, save_path=None):
    metrics = list(summary.keys())
    means = [summary[m]["mean"] for m in metrics]
    stds  = [summary[m]["std"]  for m in metrics]

    x = np.arange(len(metrics))

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.bar(x, means, yerr=stds, capsize=5)
    ax.set_xticks(x)
    ax.set_xticklabels(metrics, rotation=15)
    ax.set_ylabel("Metric value")
    ax.set_title(title or "LOSO summary (mean ± std)")
    ax.grid(axis="y", alpha=0.3)
    plt.tight_layout()

    if save_path is not None:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_path, dpi=150, bbox_inches="tight")
    else:
        plt.show()

    plt.close(fig)


def compute_ece(y_true, y_prob, n_bins=10):
    # Expected Calibration Error (ECE)
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)

    bins = np.linspace(0.0, 1.0, n_bins + 1)
    bin_ids = np.digitize(y_prob, bins) - 1

    ece = 0.0
    n = len(y_true)
    for b in range(n_bins):
        mask = bin_ids == b
        if not np.any(mask):
            continue
        prop = np.mean(mask)
        avg_conf = y_prob[mask].mean()
        avg_acc = y_true[mask].mean()
        ece += prop * abs(avg_acc - avg_conf)
    return float(ece)

# 🌳 Training Base Teacher Models (LightGBM)

This section trains and evaluates the LightGBM teacher (or general) models, including classifier calibration, temperature regression, split management, and optional plotting.

## Training & Evaluation Components

- **train_base_model**  
  Trains the base LightGBM models:  
  - Fits an `LGBMClassifier` on the training set  
  - Applies isotonic calibration using `CalibratedClassifierCV`  
  - Trains an `LGBMRegressor` on train+cal data for temperature prediction  
  Returns the calibrated classifier and regression model.

- **evaluate_base_model**  
  Evaluates a trained model on a test slice:  
  - Computes AUROC, AUPRC, Brier Score, ECE, temperature MAE  
  - Derives confusion matrix metrics (TP, FP, TN, FN)  
  - Generates optional plots (ROC, PR, Reliability)  
  - Returns a metrics dictionary and predicted probabilities.

- **train_model**  
  High-level wrapper that:  
  - Builds features/targets using `build_xy`  
  - Performs a 70/15/15 train–cal–test split  
  - Trains the base model using `train_base_model`  
  - Evaluates performance using `evaluate_base_model`  
  - Returns `(calib, reg, metrics, p_te, y_te)` preserving the original API.

## Hyperparameters
- `threshold` → hard decision cutoff for frost classification  
- `threads` → number of CPU threads for LightGBM  
- `verbose` → enables/prints detailed output  
- `plots` → controls saving ROC/PR/reliability figures  

## Key Outputs
- Calibrated probability model (`calib`)  
- Temperature regression model (`reg`)  
- Teacher metrics (AUROC, AUPRC, Brier, ECE, MAE)  
- Test-set probability predictions (`p_te`)  
- Test frost labels (`y_te`)


In [1]:
def train_base_model(
    X_tr,
    y_tr,
    X_cal,
    y_cal,
    X_trcal,
    y_temp_trcal,
    H=6,
    threads=1,
    verbose=True,
):
    """
    Train the base (general/teacher) model:
      - LGBMClassifier + isotonic calibration
      - LGBMRegressor for temperature

    This function does *no* evaluation or plotting.
    """
    # --- Classifier ---
    clf = lgb.LGBMClassifier(
        n_estimators=800,
        learning_rate=0.002,
        class_weight="balanced",
        subsample=0.8,
        colsample_bytree=0.9,
        random_state=42,
        verbose=-1,
        num_threads=threads,
    )
    clf.fit(X_tr, y_tr)

    calib = CalibratedClassifierCV(estimator=clf, method="isotonic", cv=5)
    calib.fit(X_cal, y_cal)

    # --- Regressor (train on train+cal) ---
    reg = lgb.LGBMRegressor(
        n_estimators=400,
        learning_rate=0.04,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
        num_threads=threads,
    )
    reg.fit(X_trcal, y_temp_trcal)

    return calib, reg


def evaluate_base_model(
    calib,
    reg,
    X_te,
    y_te,
    y_temp_te,
    H=6,
    threshold=0.5,
    verbose=True,
    plots=False,
    name="general",
):
    # classifier probs on test
    p_te = calib.predict_proba(X_te)[:, 1]
    y_pred = (p_te >= threshold).astype(int)

    tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    fpr       = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    tnr       = tn / (fp + tn) if (fp + tn) > 0 else 0.0

    # regression preds on test
    yhat_te = reg.predict(X_te)

    ece = compute_ece(y_te.values, p_te, n_bins=10)

    metrics = {
        "AUROC": float(roc_auc_score(y_te, p_te)),
        "AUPRC": float(average_precision_score(y_te, p_te)),
        "Brier": float(brier_score_loss(y_te, p_te)),
        "ECE": float(ece),
        "MAE_temp": float(mean_absolute_error(y_temp_te, yhat_te)),
        "threshold": float(threshold),
        "TP": int(tp),
        "FP": int(fp),
        "TN": int(tn),
        "FN": int(fn),
        "precision": float(precision),
        "recall": float(recall),
        "FPR": float(fpr),
        "TNR": float(tnr),
    }

    if verbose:
        print(f"\n========== {name.upper()} MODEL EVALUATION (H = {H}h) ==========")
    
        print("\n--- Discrimination & Temp Regression ---")
        print(f"AUROC:      {metrics['AUROC']:.4f}")
        print(f"AUPRC:      {metrics['AUPRC']:.4f}")
        print(f"MAE_temp:   {metrics['MAE_temp']:.4f}")
    
        print("\n--- Calibration Metrics ---")
        print(f"Brier:      {metrics['Brier']:.4f}")
        print(f"ECE:        {metrics['ECE']:.4f}")
    
        print("\n--- Classification Summary ---")
        print(
            f"thr={threshold:.2f} | "
            f"TP={tp} FP={fp} TN={tn} FN={fn} "
            f"(precision={precision:.3f}, recall={recall:.3f}, FPR={fpr:.3f})"
        )
    
        print(
            f"\nNext-{H}h frost probability: {p_te[-1]*100:.2f}%\n"
            f"Predicted temperature:      {yhat_te[-1]:.2f} °C"
        )
    
        print("========================================================\n")

    if plots:
        model_dir = PLOTS_ROOT / str(name)
        model_dir.mkdir(parents=True, exist_ok=True)
        title_suffix = f"{name} (H={H}h)"

        prob_dict = {str(name): p_te}

        plot_roc_curves(
            y_te,
            prob_dict,
            title=f"ROC {title_suffix}",
            save_path=model_dir / f"roc_H{H}.png",
        )
        plot_pr_curves(
            y_te,
            prob_dict,
            title=f"PR {title_suffix}",
            save_path=model_dir / f"pr_H{H}.png",
        )
        plot_reliability_curves(
            y_te,
            prob_dict,
            n_bins=10,
            title=f"Reliability {title_suffix}",
            save_path=model_dir / f"reliability_H{H}.png",
        )

    return metrics, p_te


def train_model(
    df,
    H=6,
    threshold=0.5,
    threads=1,
    verbose=True,
    plots=False,
    name="general",
):
    # --- Build features/targets ---
    X, y_frost, y_temp, feat_cols = build_xy(df, H)

    n = len(X)
    i_tr, i_cal = int(n * 0.70), int(n * 0.85)

    X_tr, y_tr = X.iloc[:i_tr],        y_frost.iloc[:i_tr]
    X_cal, y_cal = X.iloc[i_tr:i_cal], y_frost.iloc[i_tr:i_cal]
    X_te,  y_te  = X.iloc[i_cal:],     y_frost.iloc[i_cal:]
    y_temp_trcal = y_temp.iloc[:i_cal]
    y_temp_te    = y_temp.iloc[i_cal:]

    X_trcal = pd.concat([X_tr, X_cal])

    # --- TRAIN ---
    calib, reg = train_base_model(
        X_tr=X_tr,
        y_tr=y_tr,
        X_cal=X_cal,
        y_cal=y_cal,
        X_trcal=X_trcal,
        y_temp_trcal=y_temp_trcal,
        H=H,
        threads=threads,
        verbose=verbose,
    )

    # --- EVAL ---
    metrics, p_te = evaluate_base_model(
        calib=calib,
        reg=reg,
        X_te=X_te,
        y_te=y_te,
        y_temp_te=y_temp_te,
        H=H,
        threshold=threshold,
        verbose=verbose,
        plots=plots,
        name=name,
    )

    return calib, reg, metrics, p_te, y_te

# 🧑‍🎓 Weighted Ensemble Knowledge Distillation (WEKD)

This section uses all teacher models to train a student model via weighted ensemble distillation.

## WEKD Components
- **Teacher correlation penalty:**  
  Less correlated teachers receive higher influence.
- **Teacher accuracy weighting:**  
  More accurate teachers receive higher weight.
- **Softmax normalization** generates final teacher weights.
- **Ensembled teacher probabilities** are used as KD targets.

## Hyperparameters
- `alpha` → classification KD blend  
- `beta` → regression KD blend
- `threshold` → hard prediction threshold
- `verbose` → verbose outputs

## Student Outputs
- Trained KD classifier (`student_clf`)
- Trained KD temperature regressor (`student_reg`)
- Student metrics (AUROC, AUPRC, Brier, ECE, MAE)
- Plots comparing teacher ensemble vs. student

In [6]:
def _compute_teacher_weights(teachers, X_ref, y_ref, eps=1e-6):
    # Compute teacher weights using accuracy and correlation penalty.
    n_teachers = len(teachers)
    if n_teachers == 1:
        return np.array([1.0], dtype=float)

    preds = np.column_stack([t.predict_proba(X_ref)[:, 1] for t in teachers])
    hard_preds = (preds >= 0.5).astype(int)
    y_arr = y_ref.values.reshape(-1, 1)
    acc = (hard_preds == y_arr).mean(axis=0)

    corr_matrix = np.corrcoef(preds, rowvar=False)
    np.fill_diagonal(corr_matrix, 0.0)
    sum_sq_corr = np.sum(corr_matrix ** 2, axis=1)

    score = acc / (sum_sq_corr + eps)
    score_shifted = score - score.max()
    exp_score = np.exp(score_shifted)
    weights = exp_score / exp_score.sum()
    return weights


def _ensemble_probs(teachers, X, weights=None):
    preds = np.column_stack([t.predict_proba(X)[:, 1] for t in teachers])
    if weights is None:
        w = np.ones(preds.shape[1], dtype=float) / preds.shape[1]
    else:
        w = np.asarray(weights, dtype=float)
        w = w / w.sum()
    return preds @ w


def _ensemble_reg_preds(regressors, X, weights=None):
    preds = np.column_stack([r.predict(X) for r in regressors])
    if weights is None:
        w = np.ones(preds.shape[1], dtype=float) / preds.shape[1]
    else:
        w = np.asarray(weights, dtype=float)
        w = w / w.sum()
    return preds @ w

In [15]:
def train_student_kd(
    X_tr,
    y_tr,
    X_cal,
    y_cal,
    X_trcal,
    y_temp_trcal,
    teachers,
    reg_teachers,
    H=6,
    threshold=0.5,
    teacher_weights=None,
    alpha=0.7,
    beta=0.7,
    threads=1,
    verbose=True,
):
    # compute teacher weights if needed (on calibration slice)
    if teacher_weights is None:
        teacher_weights = _compute_teacher_weights(teachers, X_cal, y_cal)
        if verbose:
            print("Teacher weights (acc / sum corr^2 → softmax):", teacher_weights)

    # ---------- Classification KD ----------
    p_teacher_tr = _ensemble_probs(teachers, X_tr, teacher_weights)

    # KD targets: blend hard labels + teacher soft probs
    y_tr_kd = alpha * y_tr.values + (1.0 - alpha) * p_teacher_tr

    student_clf = lgb.LGBMRegressor(
        n_estimators=800,
        learning_rate=0.002,
        subsample=0.8,
        colsample_bytree=0.9,
        random_state=42,
        num_threads=threads,
    )
    student_clf.fit(X_tr, y_tr_kd)

    # ---------- Regression KD ----------
    temp_teacher_trcal = _ensemble_reg_preds(reg_teachers, X_trcal, teacher_weights)
    y_temp_trcal_kd = beta * y_temp_trcal.values + (1.0 - beta) * temp_teacher_trcal

    student_reg = lgb.LGBMRegressor(
        n_estimators=400,
        learning_rate=0.04,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
        num_threads=threads,
    )
    student_reg.fit(X_trcal, y_temp_trcal_kd)

    return student_clf, student_reg, teacher_weights


def evaluate_student_kd(
    student_clf,
    student_reg,
    teachers,
    reg_teachers,
    teacher_weights,
    X_te,
    y_te,
    y_temp_te,
    H=6,
    threshold=0.5,
    verbose=True,
    plots=False,
    name="student",
):
    # teacher ensemble probs on test
    p_teacher_te = _ensemble_probs(teachers, X_te, teacher_weights)

    # student probs on test
    p_te_student = student_clf.predict(X_te)
    p_te_student = np.clip(p_te_student, 0.0, 1.0)
    y_pred = (p_te_student >= threshold).astype(int)

    # regression preds on test
    yhat_te = student_reg.predict(X_te)

    # metrics
    tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    fpr       = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    tnr       = tn / (fp + tn) if (fp + tn) > 0 else 0.0

    ece_student = compute_ece(y_te.values, p_te_student, n_bins=10)
    ece_teacher = compute_ece(y_te.values, p_teacher_te, n_bins=10)

    metrics = {
        "AUROC":    float(roc_auc_score(y_te, p_te_student)),
        "AUPRC":    float(average_precision_score(y_te, p_te_student)),
        "Brier":    float(brier_score_loss(y_te, p_te_student)),
        "ECE":      float(ece_student),
        "MAE_temp": float(mean_absolute_error(y_temp_te, yhat_te)),
        "threshold": float(threshold),
        "TP": int(tp),
        "FP": int(fp),
        "TN": int(tn),
        "FN": int(fn),
        "precision": float(precision),
        "recall": float(recall),
        "FPR": float(fpr),
        "TNR": float(tnr),
        "teacher_weights": teacher_weights.tolist(),
    }

    if verbose:
        print(f"\n========== STUDENT MODEL EVALUATION (H = {H}h) ==========")
    
        print("\n--- Discrimination & Temp Regression ---")
        print(f"AUROC:      {metrics['AUROC']:.4f}")
        print(f"AUPRC:      {metrics['AUPRC']:.4f}")
        print(f"MAE_temp:   {metrics['MAE_temp']:.4f}")
    
        print("\n--- Calibration Metrics ---")
        print(f"Brier:      {metrics['Brier']:.4f}")
        print(f"ECE:        {metrics['ECE']:.4f}")
    
        print("\n--- Classification Summary ---")
        print(
            f"thr={threshold:.2f} | "
            f"TP={tp} FP={fp} TN={tn} FN={fn} "
            f"(precision={precision:.3f}, recall={recall:.3f}, FPR={fpr:.3f})"
        )
    
        print(
            f"\nNext-{H}h frost probability: {p_te_student[-1]*100:.2f}%\n"
            f"Predicted temperature:      {yhat_te[-1]:.2f} °C"
        )

        print("=========================================================\n")

    if plots:
        model_dir = PLOTS_ROOT / str(name)
        model_dir.mkdir(parents=True, exist_ok=True)
        title_suffix = f"{name} (H={H}h)"

        prob_dict = {
            "TeacherEnsemble": p_teacher_te,
            "Student": p_te_student,
        }

        plot_roc_curves(
            y_te,
            prob_dict,
            title=f"ROC {title_suffix}",
            save_path=model_dir / f"roc_H{H}.png",
        )
        plot_pr_curves(
            y_te,
            prob_dict,
            title=f"PR {title_suffix}",
            save_path=model_dir / f"pr_H{H}.png",
        )
        plot_reliability_curves(
            y_te,
            prob_dict,
            n_bins=10,
            title=f"Reliability {title_suffix}",
            save_path=model_dir / f"reliability_H{H}.png",
        )

        auprc_teacher = float(average_precision_score(y_te, p_teacher_te))
        brier_teacher = float(brier_score_loss(y_te, p_teacher_te))

        plot_metric_bar(
            "AUPRC",
            {"TeacherEnsemble": auprc_teacher, "Student": metrics["AUPRC"]},
            title=f"AUPRC {title_suffix}",
            save_path=model_dir / f"auprc_H{H}.png",
        )
        plot_metric_bar(
            "Brier",
            {"TeacherEnsemble": brier_teacher, "Student": metrics["Brier"]},
            title=f"Brier {title_suffix}",
            save_path=model_dir / f"brier_H{H}.png",
        )
        plot_metric_bar(
            "ECE",
            {"TeacherEnsemble": ece_teacher, "Student": metrics["ECE"]},
            title=f"ECE {title_suffix}",
            save_path=model_dir / f"ece_H{H}.png",
        )

    return metrics, p_te_student, p_teacher_te


def train_student_from_ensemble(
    df,
    teachers,
    reg_teachers,
    H=6,
    threshold=0.5,
    teacher_weights=None,
    alpha=0.7,
    beta=0.7,
    threads=1,
    verbose=True,
    plots=False,
    name="student",
):
    # --- Build features/targets ---
    X, y_frost, y_temp, feat_cols = build_xy(df, H)

    n = len(X)
    i_tr, i_cal = int(n * 0.70), int(n * 0.85)

    X_tr, y_tr = X.iloc[:i_tr],        y_frost.iloc[:i_tr]
    X_cal, y_cal = X.iloc[i_tr:i_cal], y_frost.iloc[i_tr:i_cal]
    X_te,  y_te  = X.iloc[i_cal:],     y_frost.iloc[i_cal:]
    y_temp_trcal = y_temp.iloc[:i_cal]
    y_temp_te    = y_temp.iloc[i_cal:]

    X_trcal = pd.concat([X_tr, X_cal])

    # --- TRAIN ---
    student_clf, student_reg, teacher_weights = train_student_kd(
        X_tr=X_tr,
        y_tr=y_tr,
        X_cal=X_cal,
        y_cal=y_cal,
        X_trcal=X_trcal,
        y_temp_trcal=y_temp_trcal,
        teachers=teachers,
        reg_teachers=reg_teachers,
        H=H,
        threshold=threshold,
        teacher_weights=teacher_weights,
        alpha=alpha,
        beta=beta,
        threads=threads,
        verbose=verbose,
    )

    # --- EVAL ---
    metrics, p_te_student, p_teacher_te = evaluate_student_kd(
        student_clf=student_clf,
        student_reg=student_reg,
        teachers=teachers,
        reg_teachers=reg_teachers,
        teacher_weights=teacher_weights,
        X_te=X_te,
        y_te=y_te,
        y_temp_te=y_temp_te,
        H=H,
        threshold=threshold,
        verbose=verbose,
        plots=plots,
        name=name,
    )

    return student_clf, student_reg, metrics, p_te_student, p_teacher_te, y_te

# 🚀 Main Pipeline: General & Student Model Execution

This final section executes the full training procedure.

## Pipeline Steps
- Train a general LightGBM model on all stations combined
- Train all teacher models per station
- Train student model using Weighted Ensemble Knowledge Distillation (WEKD)
- Save all models and plots
- Clean RAM between horizons

In [8]:
# To Clear Model and Plot Directory
# !rm -rf models/*
# !rm -rf plots/*

In [9]:
from pathlib import Path
import joblib
import os
import gc

datasets = clean_datasets(get_datasets()) # Dictionary of CIMIS Datasets
stations = list(datasets.keys()) # Station Names
df_all = pd.concat(list(datasets.values()), ignore_index=True) # DataFrame with All Stations Data

metrics_over_H = {}  # {model_name: {H: metrics_dict}}

# HyperParams
tfs = [3, 6, 12, 24] # TimeFrames
threshold = 0.3 # Threshold for Models
thread_per_model = 16 # Threads Per Model
metrics_to_plot = ["Brier", "ECE", "AUPRC", "AUROC"]  # Metrics to Plot
verbose = True # Verbose Output
save_models = True # Save Models
save_plots = True # Save Plots

In [ ]:
GENERAL_DIR = MODELS_ROOT / "general"
GENERAL_DIR.mkdir(parents=True, exist_ok=True)

# Track metrics across horizons
if "general" not in metrics_over_H:
    metrics_over_H["general"] = {}

print(f"\n Training General Model...")
for tf in tfs:
    calib, reg, general_metrics, p_te_gen, y_te_gen = train_model(
        df=df_all, 
        H=tf, 
        threshold=threshold,
        threads=thread_per_model, 
        verbose=verbose,
        plots=save_plots,        
        name="general"
    )

    general_calib_path = GENERAL_DIR / f"H{tf}_calib.joblib"
    general_reg_path   = GENERAL_DIR / f"H{tf}_reg.joblib"

    metrics_over_H["general"][tf] = general_metrics

    if save_models:
        joblib.dump(calib, general_calib_path)
        joblib.dump(reg,   general_reg_path)

    # Delete All References and Garbage Collect [Memory Clean Up]
    del calib, reg
    gc.collect()

if save_plots:
    plot_horizon_summaries(metrics_over_H, metrics_to_plot)
    metrics_over_H = {} # Reset Dictionary

In [ ]:
# Create Directories for Teachers
TEACHERS_DIR = {}
for station in stations:
    station_dir = MODELS_ROOT / station
    station_dir.mkdir(parents=True, exist_ok=True)
    TEACHERS_DIR[station] = station_dir
    
# Create Directory for Student
STUDENTS_DIR = MODELS_ROOT / "students"
STUDENTS_DIR.mkdir(parents=True, exist_ok=True)

for tf in tfs:
    # Ensemble of Teachers
    teachers = []
    reg_teachers = []

    # Train, Test, Plot, then Dump Teacher Models
    for station in stations:
        print(f"\nTraining {station} (H={tf})...")

        calib, reg, metrics, p_te, y_te = train_model(
            df=datasets[station],
            H=tf,
            threshold=threshold,
            threads=thread_per_model,
            verbose=verbose,
            plots=save_plots,                   
            name=station,                 
        )

        teachers.append(calib)
        reg_teachers.append(reg)

        # Track metrics across horizons
        if station not in metrics_over_H:
            metrics_over_H[station] = {}
        metrics_over_H[station][tf] = metrics

        if save_models:
            joblib.dump(calib, TEACHERS_DIR[station] / f"H{tf}_calib.joblib")
            joblib.dump(reg,   TEACHERS_DIR[station] / f"H{tf}_reg.joblib")

        del p_te, y_te

    # Train, Test, Plot, then Dump Student Model
    print(f"\nTraining Student from Teachers (H={tf})...")
    
    student_clf, student_reg, student_metrics, p_te_student, p_teacher_te, y_te_student = train_student_from_ensemble(
        df=df_all,
        teachers=teachers,
        reg_teachers=reg_teachers,
        H=tf,
        threshold=threshold,
        threads=thread_per_model,
        alpha=0.7,
        beta=0.7,
        verbose=verbose,
        plots=save_plots,    
        name="student",
    )

    # Track metrics across horizons
    if "student" not in metrics_over_H:
        metrics_over_H["student"] = {}
    metrics_over_H["student"][tf] = student_metrics
    
    if save_models:
        joblib.dump(student_clf, STUDENTS_DIR / f"H{tf}_calib.joblib")
        joblib.dump(student_reg, STUDENTS_DIR / f"H{tf}_reg.joblib")

    # Delete All References and Garbage Collect [Memory Clean Up]
    del teachers, reg_teachers
    del student_clf, student_reg, p_te_student, p_teacher_te, y_te_student
    gc.collect()

if save_plots:
    plot_horizon_summaries(metrics_over_H, metrics_to_plot)
    metrics_over_H = {} # Reset Dictionary

# 📍 Spatial Generalization with Leave-One-Station-Out (LOSO)

Leave-One-Station-Out (LOSO) evaluates how well models generalize to **unseen geographic locations** by repeatedly holding out one station for testing while training on all others.

## What LOSO Does
- Select one station as the held-out test set  
- Train on all remaining stations  
- Evaluate only on the held-out station  
- Repeat for every station  

**Outputs:**  
Per-station metrics, performance distributions, and mean ± std summaries showing spatial robustness.

## Why LOSO Matters
Random or temporal splits cannot detect spatial overfitting. LOSO answers:
- Does the model memorize station-specific microclimates?  
- Can it generalize to unseen locations?  
- Does knowledge distillation improve spatial transfer?  
- How does the general model compare to the student across geography?

In [12]:
def run_loso_general(
    datasets,
    H=6,
    threshold=0.5,
    threads=1,
    verbose=True,
    plots=False,
    name="general",
):
    stations = list(datasets.keys())
    per_station_metrics = {}

    loso_dir = PLOTS_ROOT / "LOSO" / f"{name}_H{H}"
    if plots:
        loso_dir.mkdir(parents=True, exist_ok=True)

    # LOOP OVER STATIONS (Leave-One-Station-Out)
    for held_out in stations:
        if verbose:
            print("\n" + "=" * 70)
            print(f"   LOSO GENERAL: Held-out station '{held_out}' (H={H})")
            print("=" * 70)

        # Build LOSO training & test sets
        df_train = pd.concat(
            [df for s, df in datasets.items() if s != held_out],
            ignore_index=True
        )
        df_test = datasets[held_out]

        # Build XY for train split
        X_all, y_frost_all, y_temp_all, _ = build_xy(df_train, H)

        n_all = len(X_all)
        i_tr, i_cal = int(0.70 * n_all), int(0.85 * n_all)

        X_tr, y_tr       = X_all.iloc[:i_tr],       y_frost_all.iloc[:i_tr]
        X_cal, y_cal     = X_all.iloc[i_tr:i_cal],  y_frost_all.iloc[i_tr:i_cal]
        X_te_train       = X_all.iloc[i_cal:]
        y_te_train       = y_frost_all.iloc[i_cal:]
        y_temp_trcal     = y_temp_all.iloc[:i_cal]
        y_temp_te_train  = y_temp_all.iloc[i_cal:]
        X_trcal = pd.concat([X_tr, X_cal])

        # Train base general model on df_train only
        calib, reg = train_base_model(
            X_tr=X_tr,
            y_tr=y_tr,
            X_cal=X_cal,
            y_cal=y_cal,
            X_trcal=X_trcal,
            y_temp_trcal=y_temp_trcal,
            H=H,
            threads=threads,
            verbose=verbose,
        )

        # Evaluate on HELD-OUT station
        X_te, y_frost_te, y_temp_te, _ = build_xy(df_test, H)

        metrics, p_te = evaluate_base_model(
            calib=calib,
            reg=reg,
            X_te=X_te,
            y_te=y_frost_te,
            y_temp_te=y_temp_te,
            H=H,
            threshold=threshold,
            verbose=verbose,
            plots=False,  # avoid spam
            name=f"LOSO_general_excluding_{held_out}",
        )

        # Save metrics for this station
        per_station_metrics[held_out] = metrics

        # Drop big dataframes and arrays
        del df_train, df_test
        del X_all, y_frost_all, y_temp_all
        del X_tr, y_tr, X_cal, y_cal, X_te_train, y_te_train
        del y_temp_trcal, y_temp_te_train, X_trcal
        del X_te, y_frost_te, y_temp_te
        # Drop models and predictions for this fold
        del calib, reg, p_te

        gc.collect()

    # Compute LOSO SUMMARY (mean ± std) in two metric groups
    group_A = ["AUROC", "AUPRC", "MAE_temp"]   # discrimination & regression
    group_B = ["Brier", "ECE"]                 # calibration metrics

    summary = {"group_A": {}, "group_B": {}}

    for m in group_A:
        vals = np.array([per_station_metrics[s][m] for s in stations], dtype=float)
        summary["group_A"][m] = {
            "mean": float(vals.mean()),
            "std":  float(vals.std()),
        }

    for m in group_B:
        vals = np.array([per_station_metrics[s][m] for s in stations], dtype=float)
        summary["group_B"][m] = {
            "mean": float(vals.mean()),
            "std":  float(vals.std()),
        }

    # Print summary
    if verbose:
        print("\n========== LOSO GENERAL SUMMARY (Mean ± Std) ==========")

        print("\n--- Discrimination & Temp Regression ---")
        for m in group_A:
            print(f"{m}: {summary['group_A'][m]['mean']:.4f} ± {summary['group_A'][m]['std']:.4f}")

        print("\n--- Calibration Metrics ---")
        for m in group_B:
            print(f"{m}: {summary['group_B'][m]['mean']:.4f} ± {summary['group_B'][m]['std']:.4f}")

        print("=======================================================\n")

    # Summary plots
    if plots:
        # Main summary: AUROC, AUPRC, MAE_temp
        plot_loso_summary(
            summary["group_A"],
            title=f"LOSO General Model Summary – Discrimination & Temp (H={H})",
            save_path=loso_dir / f"summary_LOSO_general_main_H{H}.png",
        )

        # Calibration-only summary: Brier, ECE
        plot_loso_summary(
            summary["group_B"],
            title=f"LOSO General Model Summary – Calibration (H={H})",
            save_path=loso_dir / f"summary_LOSO_general_calib_H{H}.png",
        )

    return per_station_metrics, summary

In [11]:
def run_loso_student(
    datasets,
    teachers_by_station,
    reg_teachers_by_station,
    H=6,
    threshold=0.5,
    alpha=0.7,
    beta=0.7,
    threads=1,
    verbose=True,
    plots=False,
    name="student",
):
    stations = list(datasets.keys())
    per_station_metrics = {}

    # Where to save LOSO summaries and plots
    loso_dir =  PLOTS_ROOT / "LOSO" / f"{name}_H{H}"
    if plots:
        loso_dir.mkdir(parents=True, exist_ok=True)

    for held_out in stations:
        if verbose:
            print("\n" + "="*70)
            print(f"   LOSO Fold – Held-out station: {held_out} (H={H})")
            print("="*70)

        # Build LOSO training data
        df_train = pd.concat(
            [df for s, df in datasets.items() if s != held_out],
            ignore_index=True
        )
        df_test = datasets[held_out]

        # Teachers used for this fold = all except held-out
        fold_stations = [s for s in stations if s != held_out]
        fold_teachers     = [teachers_by_station[s] for s in fold_stations]
        fold_reg_teachers = [reg_teachers_by_station[s] for s in fold_stations]

        # Build XY for the TRAIN dataset (for computing teacher weights)
        X_all, y_frost_all, y_temp_all, _ = build_xy(df_train, H)
        n_all = len(X_all)
        i_tr, i_cal = int(0.70 * n_all), int(0.85 * n_all)
        X_tr, y_tr   = X_all.iloc[:i_tr],       y_frost_all.iloc[:i_tr]
        X_cal, y_cal = X_all.iloc[i_tr:i_cal], y_frost_all.iloc[i_tr:i_cal]
        X_te_train   = X_all.iloc[i_cal:]
        y_te_train   = y_frost_all.iloc[i_cal:]
        y_temp_trcal = y_temp_all.iloc[:i_cal]
        y_temp_te_train = y_temp_all.iloc[i_cal:]
        X_trcal = pd.concat([X_tr, X_cal])

        # Compute teacher weights for this LOSO fold
        teacher_weights = _compute_teacher_weights(fold_teachers, X_cal, y_cal)

        # Train a WEKD student on df_train only
        student_clf, student_reg, teacher_weights = train_student_kd(
            X_tr=X_tr,
            y_tr=y_tr,
            X_cal=X_cal,
            y_cal=y_cal,
            X_trcal=X_trcal,
            y_temp_trcal=y_temp_trcal,
            teachers=fold_teachers,
            reg_teachers=fold_reg_teachers,
            H=H,
            threshold=threshold,
            teacher_weights=teacher_weights,
            alpha=alpha,
            beta=beta,
            threads=threads,
            verbose=verbose,
        )

        # Evaluate student on HELD-OUT station only
        X_te, y_frost_te, y_temp_te, _ = build_xy(df_test, H)

        metrics, p_student, p_teacher = evaluate_student_kd(
            student_clf=student_clf,
            student_reg=student_reg,
            teachers=fold_teachers,
            reg_teachers=fold_reg_teachers,
            teacher_weights=teacher_weights,
            X_te=X_te,
            y_te=y_frost_te,
            y_temp_te=y_temp_te,
            H=H,
            threshold=threshold,
            verbose=verbose,
            plots=False,   # avoid too many plots during LOSO
            name=f"LOSO_student_excluding_{held_out}",
        )

        # Save per-station metrics
        per_station_metrics[held_out] = metrics

        # Big dataframes and arrays
        del df_train, df_test
        del X_all, y_frost_all, y_temp_all
        del X_tr, y_tr, X_cal, y_cal, X_te_train, y_te_train
        del y_temp_trcal, y_temp_te_train, X_trcal
        del X_te, y_frost_te, y_temp_te
        # Models and predictions
        del student_clf, student_reg
        del p_student, p_teacher
        del fold_teachers, fold_reg_teachers
        del teacher_weights

        gc.collect()

    # Metric groups (due to different numeric scales)
    group_A = ["AUROC", "AUPRC", "MAE_temp"]   # discrimination & regression
    group_B = ["Brier", "ECE"]                 # calibration metrics
    
    summary = {"group_A": {}, "group_B": {}}
    
    # Compute per-group means + std
    for m in group_A:
        vals = np.array([per_station_metrics[s][m] for s in stations], dtype=float)
        summary["group_A"][m] = {
            "mean": float(vals.mean()),
            "std":  float(vals.std()),
        }
    
    for m in group_B:
        vals = np.array([per_station_metrics[s][m] for s in stations], dtype=float)
        summary["group_B"][m] = {
            "mean": float(vals.mean()),
            "std":  float(vals.std()),
        }
    
    # Print summary
    if verbose:
        print("\n\n========== LOSO SUMMARY (Mean ± Std) ==========")
    
        print("\n--- Discrimination & Temp Regression ---")
        for m in group_A:
            print(f"{m}: {summary['group_A'][m]['mean']:.4f} ± {summary['group_A'][m]['std']:.4f}")
    
        print("\n--- Calibration Metrics ---")
        for m in group_B:
            print(f"{m}: {summary['group_B'][m]['mean']:.4f} ± {summary['group_B'][m]['std']:.4f}")
    
        print("==============================================\n")
    
    # Summary plots
    if plots:
        # Main summary: discrimination & temp regression (AUROC, AUPRC, MAE_temp)
        plot_loso_summary(
            summary["group_A"],
            title=f"LOSO WEKD Student Summary – Discrimination & Temp (H={H})",
            save_path=loso_dir / f"summary_LOSO_main_H{H}.png",
        )
    
        # Calibration-only summary: Brier, ECE
        plot_loso_summary(
            summary["group_B"],
            title=f"LOSO WEKD Student Summary – Calibration (H={H})",
            save_path=loso_dir / f"summary_LOSO_calib_H{H}.png",
        )

    return per_station_metrics, summary

# 🌍 LOSO Pipeline: Spatial Generalization

The LOSO pipeline tests how models generalize to unseen stations by training on all but one station and evaluating on the held-out location. This process repeats for every station.

## Pipwline Steps
- Hold out one station as test data  
- Train on all remaining stations  
- Evaluate only on the held-out station  
- Repeat for all stations  
- Compute mean ± std across metrics  
- Produce two plots:  
  - Main (AUROC, AUPRC, MAE_temp)  
  - Calibration (Brier, ECE)

In [ ]:
for tf in tfs:
    run_loso_general(
        datasets,
        H=tf,
        threshold=threshold,
        threads=thread_per_model,
        verbose=verbose,
        plots=save_plots, # makes LOSO summary plot with mean ± std
        name="general",
    )

In [ ]:
# Store LOSO results for later analysis / tables
loso_student_results = {}  # H -> {"per_station": ..., "summary": ...}

for tf in tfs:
    print(f"\n=== Running LOSO for student (H={tf}) ===")

    # Load teachers for this horizon
    teachers_by_station = {}
    reg_teachers_by_station = {}

    for station in stations:
        calib_path = TEACHERS_DIR[station] / f"H{tf}_calib.joblib"
        reg_path   = TEACHERS_DIR[station] / f"H{tf}_reg.joblib"

        teachers_by_station[station]     = joblib.load(calib_path)
        reg_teachers_by_station[station] = joblib.load(reg_path)

    # Run LOSO for this horizon
    per_station_metrics, summary = run_loso_student(
        datasets=datasets,
        teachers_by_station=teachers_by_station,
        reg_teachers_by_station=reg_teachers_by_station,
        H=tf,
        threshold=threshold,
        alpha=0.7,
        beta=0.7,
        threads=thread_per_model,
        verbose=verbose,
        plots=save_plots,  # makes LOSO summary plot with mean ± std
        name=f"student",
    )

    # Save results by horizon
    loso_student_results[tf] = {
        "per_station": per_station_metrics,
        "summary": summary,
    }

    # Memory Cleanup (safe version)

    # Remove per-station teachers to prevent accumulation
    del teachers_by_station
    del reg_teachers_by_station

    # Remove LOSO metrics dicts from RAM if you want
    # (but KEEP loso_student_results — that’s your output)
    # del per_station_metrics
    # del summary

    gc.collect()